## Transformation between ground/obc asp_l1 and on-board telemetry ##

In [1]:
import numpy as np
from Quaternion import Quat
from mica.archive import asp_l1
from astropy.table import Table
from kadi import events
from Ska.engarchive import fetch as fetch

* <code>aca_misalign</code>

In [2]:
calfiles = ['/data/aca/archive/asp1/08/08008_v03/pcadf284100553N003_acal1.fits.gz']
acal1 = Table.read(calfiles[-1])
aca_misalign = acal1['aca_misalign'][0]

* OBC <code>asp_l1</code> aspect solution

In [3]:
# Get OBC solution, convert using aca_misalign to obtain quat
#arc5gl = Ska.arc5gl.Arc5gl()
#arc5gl.sendline('obsid=8008')
#arc5gl.sendline('version=last')
#arc5gl.sendline('get asp1{obcsol}')
#del arc5gl

In [4]:
obc_sol = Table.read('pcadf284099196N002_osol1.fits.gz')

* Grnd <code>asp_l1</code> aspect solution

In [5]:
solfiles = asp_l1.get_files(obsid=8008, revision='last', content='ASPSOL')
print(solfiles)
grnd_sol = Table.read(solfiles[-1])

['/data/aca/archive/asp1/08/08008_v03/pcadf284100553N003_asol1.fits.gz']


* Match times

In [6]:
# on-board telemetry
obsid = 8008
d = events.dwells.filter(obsid=obsid)[0]
atts = fetch.MSIDset(['aoattqt*'], d.start, d.stop)
quat = Quat([atts['aoattqt{}'.format(i + 1)].vals[0] for i in range(4)])
atts['aoattqt1'].times[0]

284100553.48501438

In [7]:
grnd_sol['time'][0]

284100553.74126387

In [8]:
obc_sol['time'][1324]

284100553.48501384

* Quaternions

In [11]:
quat

<Quat q1=0.14960202 q2=0.49089972 q3=0.83147544 q4=0.21280339>

In [12]:
Quat(grnd_sol['q_att'][0])

<Quat q1=0.15019172 q2=0.49235635 q3=0.83060600 q4=0.21241785>

In [13]:
Quat(obc_sol['q_att'][1324])

<Quat q1=0.15009502 q2=0.49197906 q3=0.83082895 q4=0.21248847>

* Transformation

In [14]:
# We want to match the on-board quaternion
quat

<Quat q1=0.14960202 q2=0.49089972 q3=0.83147544 q4=0.21280339>

In [15]:
Ts_obc = Quat(obc_sol['q_att'][1324]).transform
Ts_grnd = Quat(grnd_sol['q_att'][0]).transform

In [16]:
q_obc_trans = Quat(np.dot(aca_misalign, Ts_obc.T)).inv()
q_grnd_trans = Quat(np.dot(aca_misalign, Ts_grnd.T)).inv()

In [17]:
q_obc_trans, q_grnd_trans

(<Quat q1=0.14950525 q2=0.49052201 q3=0.83169768 q4=0.21287392>,
 <Quat q1=0.14960202 q2=0.49089972 q3=0.83147544 q4=0.21280339>)